In [9]:
import pandas as pd
import numpy as np

# !pip install deepctr[gpu]

# 导入deepctr
from deepctr.models import DIN,DIEN,BST
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import * 
import tensorflow as tf

from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

generate='./data/generate/'
path='./data/rec_data/'

In [10]:
def fun(x):
    if x<=200:
        return 1
    elif x<=1000:
        return 2
    elif x<=5000:
        return 3
    else:
        return 4

In [11]:
user_seq=pd.read_csv(generate+'user_seq_raw.csv',usecols=['suv','itemId'])
user_seq['item_count'] = user_seq.groupby('itemId')['suv'].transform('count')
user_seq['level']= user_seq['item_count'].apply(lambda x: fun(x))
del user_seq['item_count']

level=user_seq[['itemId','level']]
level.drop_duplicates(inplace=True)
del user_seq['level']

train=pd.read_csv(path+'train-dataset.csv')
test=pd.read_csv(path+'test-dataset.csv')
test.rename(columns={'testSampleId':'sampleId'},inplace=True)

category=pd.read_csv(generate+'category.csv')
print(len(category))
category.rename(columns={"id":"itemId"},inplace=True)
category=pd.merge(category,level,how='left',on='itemId')
print(category)

train=pd.concat([train,test])
del test
train=pd.concat([train,user_seq])
train=pd.merge(train,category,how='left',on='itemId')

/tmp/ipykernel_64152/3692499866.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level.drop_duplicates(inplace=True)


146791
          itemId  category  level
0       10000000        12    1.0
1       10000001         1    1.0
2       10000002         4    NaN
3       10000003         0    1.0
4       10000004         1    1.0
...          ...       ...    ...
146786  10146786         6    1.0
146787  10146787         3    1.0
146788  10146788        12    NaN
146789  10146789         4    1.0
146790  10146790        12    1.0

[146791 rows x 3 columns]


In [12]:
# 数据处理sparse、dense
for feat in ['itemId']:
    lbe = LabelEncoder()
    train[feat] = lbe.fit_transform(train[feat])

# train[['itemId']]+=1
# train[['category']]+=1
    
ygq=train.copy()
user_seq=train[train['sampleId'].isna()]
user_seq=user_seq[['suv','itemId','category','level']]
train=train[train['sampleId'].notna()]

In [13]:
# 造点统计特征
train['pvid_count'] = train.groupby('pvId')['itemId'].transform('count')
train['pvid_item_nunique'] = train.groupby('pvId')['itemId'].transform('nunique')
train['pvid_suv_nunique'] = train.groupby('pvId')['suv'].transform('nunique')
train['item_count'] = train.groupby('itemId')['itemId'].transform('count')
train['item_suv_nunique'] = train.groupby('pvId')['suv'].transform('nunique')

In [14]:
# train=train[train['userSeq'].notna()]
del train['userSeq']
del train['logTs']

In [15]:
hist_click=user_seq[['suv','itemId','category','level']].groupby(['suv']).agg({list}).reset_index()
his_behavior_df=pd.DataFrame()
his_behavior_df['suv']=hist_click['suv']
his_behavior_df['hist_itemId']=hist_click['itemId']
his_behavior_df['hist_category']=hist_click['category']
his_behavior_df['hist_level']=hist_click['level']
del hist_click
del user_seq

In [16]:
data=pd.merge(train,his_behavior_df,how='left',on='suv')

In [17]:
tmps=data['hist_itemId'].values.tolist()
for i in range(len(tmps)):
    if not isinstance(tmps[i],list):
        tmps[i]=[0]
data['hist_itemId']=tmps

tmps=data['hist_category'].values.tolist()
for i in range(len(tmps)):
    if not isinstance(tmps[i],list):
        tmps[i]=[0]
data['hist_category']=tmps

tmps=data['hist_level'].values.tolist()
for i in range(len(tmps)):
    if not isinstance(tmps[i],list):
        tmps[i]=[0]
data['hist_level']=tmps

# data['length'].fillna(0,inplace=True)

del data['sampleId']

In [18]:
# 把特征分开
sparse_fea = ['suv', 'pvId','itemId', 'operator','category','level',
              'browserType', 'deviceType', 'osType', 'province', 'city']

behavior_fea = ['itemId','category','level']

hist_behavior_fea = ['hist_itemId','hist_category','hist_level']

dense_fea = ['pvid_count', 'pvid_item_nunique', 'pvid_suv_nunique', 'item_count', 'item_suv_nunique']

In [19]:
sparse_fea_tmp = ['suv', 'pvId', 'operator',
              'browserType', 'deviceType', 'osType', 'province', 'city']
# 数据处理sparse、dense
for feat in sparse_fea_tmp:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

data[dense_fea] = data[dense_fea].fillna(0, )
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_fea] = mms.fit_transform(data[dense_fea])

In [20]:
data.to_feather(generate+'DIN_data_level.feather')
ygq.to_feather(generate+'DIN_data_ygq_level.feather')

============================================================================================================

## 上为数据处理，下为输入模型

In [1]:
import pandas as pd
import numpy as np

# !pip install deepctr[gpu]

# 导入deepctr
from deepctr.models import DIN,DIEN,BST
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import * 
import tensorflow as tf

from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

generate='./data/generate/'
path='./data/rec_data/'

2022-05-16 17:18:31.929868: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# 把特征分开
sparse_fea = ['suv', 'pvId','itemId', 'operator','category','level',
              'browserType', 'deviceType', 'osType', 'province', 'city']

behavior_fea = ['itemId','category','level']

hist_behavior_fea = ['hist_itemId','hist_category','hist_level']

dense_fea = ['pvid_count', 'pvid_item_nunique', 'pvid_suv_nunique', 'item_count', 'item_suv_nunique']

In [3]:
data=pd.read_feather(generate+'DIN_data_level.feather')
ygq=pd.read_feather(generate+'DIN_data_ygq_level.feather')

In [4]:
emb_dim=32
max_len=32

sparse_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique() + 1, embedding_dim=emb_dim) for feat in sparse_fea]
dense_feature_columns = [DenseFeat(feat, 1, ) for feat in dense_fea]

var_feature_columns = [VarLenSparseFeat(SparseFeat(feat, vocabulary_size=ygq[feat.split('_')[1]].nunique() + 1,
                        embedding_dim=emb_dim, embedding_name=feat.split('_')[1]), maxlen=max_len) for feat in hist_behavior_fea]

dnn_feature_columns = sparse_feature_columns + dense_feature_columns + var_feature_columns


In [5]:
train=data[data['label'].notna()]
test=data[data['label'].isna()]
del data
del ygq
test.head(20)

,label,pvId,suv,itemId,operator,browserType,deviceType,osType,province,city,category,level,pvid_count,pvid_item_nunique,pvid_suv_nunique,item_count,item_suv_nunique,hist_itemId,hist_category,hist_level
4233804,NaN,5,319111,89718,258,26,0,8,32,186,2,3.0,0.041667,0.045016,0.0,0.052617,0.0,[0],[0],[0.0]
4233805,NaN,5,319111,96837,258,26,0,8,32,186,6,3.0,0.041667,0.045016,0.0,0.353261,0.0,[0],[0],[0.0]
4233806,NaN,5,319111,89845,258,26,0,8,32,186,7,4.0,0.041667,0.045016,0.0,1.000000,0.0,[0],[0],[0.0]
4233807,NaN,5,319111,97642,258,26,0,8,32,186,3,3.0,0.041667,0.045016,0.0,0.142227,0.0,[0],[0],[0.0]
4233808,NaN,5,319111,84056,258,26,0,8,32,186,4,4.0,0.041667,0.045016,0.0,0.593296,0.0,[0],[0],[0.0]
4233809,NaN,5,319111,76837,258,26,0,8,32,186,7,4.0,0.041667,0.045016,0.0,0.227692,0.0,[0],[0],[0.0]
4233810,NaN,5,319111,97231,258,26,0,8,32,186,7,1.0,0.041667,0.045016,0.0,0.010952,0.0,[0],[0],[0.0]
4233811,NaN,5,319111,87005,258,26,0,8,32,186,2,3.0,0.041667,0.045016,0.0,0.133669,0.0,[0],[0],[0.0]
4233812,NaN,5,319111,90927,258,26,0,8,32,186,8,3.0,0.041667,0.045016,0.0,0.289557,0.0,[0],[0],[0.0]
4233813,NaN,5,319111,93330,258,26,0,8,32,186,6,3.0,0.041667,0.045016,0.0,0.372753,0.0,[0],[0],[0.0]


In [6]:
x_trn = {}
for name in get_feature_names(dnn_feature_columns):
    if name in hist_behavior_fea:
        # 这是历史行为序列
        his_list = [l for l in train[name]]
        x_trn[name] = pad_sequences(his_list, maxlen=max_len, padding='post')      # 二维数组
    else:
        x_trn[name] = train[name].values

x_tst = {}
for name in get_feature_names(dnn_feature_columns):
    if name in hist_behavior_fea:
        # 这是历史行为序列
        his_list = [l for l in test[name]]
        x_tst[name] = pad_sequences(his_list, maxlen=max_len, padding='post')      # 二维数组
    else:
        x_tst[name] = test[name].values

In [7]:
y_trn=train['label'].values
y_tst=test['label'].values
# y_val=val['label'].values

In [8]:
# 建立模型
model = DIN(dnn_feature_columns, behavior_fea)

# 查看模型结构
model.summary()

# 模型编译
model.compile('adam', 'binary_crossentropy',metrics=['binary_crossentropy','accuracy'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
suv (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
pvId (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
itemId (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
operator (InputLayer)           [(None, 1)]          0                                            
______________________________________________________________________________________________

In [9]:
history = model.fit(x_trn, y_trn, verbose=1,epochs=3,batch_size=256)

Train on 4233804 samples


2022-05-16 17:19:30.649086: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-16 17:19:30.812852: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 0 with properties: 
name: NVIDIA RTX A5000 major: 8 minor: 6 memoryClockRate(GHz): 1.695
pciBusID: 0000:41:00.0
2022-05-16 17:19:30.812875: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-05-16 17:19:30.816445: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-05-16 17:19:30.818013: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022-05-16 17:19:30.818221: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcurand.so.10
2022-05-16 17:19:30.818644: I tensorflow/stream_executor/platform/defaul

Epoch 1/3


2022-05-16 17:19:33.685319: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-05-16 17:19:35.058804: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


4233804/4233804 [==============================] - 236s 56us/sample - loss: 0.4696 - binary_crossentropy: 0.4590 - acc: 0.8133
Epoch 2/3
4233804/4233804 [==============================] - 237s 56us/sample - loss: 0.4548 - binary_crossentropy: 0.4381 - acc: 0.8211
Epoch 3/3
4233804/4233804 [==============================] - 232s 55us/sample - loss: 0.4474 - binary_crossentropy: 0.4258 - acc: 0.8258


In [10]:
# 模型预测
test['rate'] = model.predict(x_tst, verbose=1, batch_size=256)
test[['suv', 'itemId', 'rate']].to_csv('./din_rank_score.csv', index=False)

   5888/1422218 [..............................] - ETA: 56s 

2022-05-16 17:31:19.339343: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:697] Iteration = 0, topological sort failed with message: The graph couldn't be sorted in topological order.
2022-05-16 17:31:19.340249: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:697] Iteration = 1, topological sort failed with message: The graph couldn't be sorted in topological order.
2022-05-16 17:31:19.341161: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:568] layout failed: Invalid argument: The graph couldn't be sorted in topological order.
2022-05-16 17:31:19.342700: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:568] model_pruner failed: Invalid argument: MutableGraphView::MutableGraphView error: node 'attention_sequence_pooling_layer/local_activation_unit/concat' has self cycle fanin 'attention_sequence_pooling_layer/local_activation_unit/concat'.
2022-05-16 17:31:19.346825: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:568] remapper failed

1422218/1422218 [==============================] - 26s 18us/sample


In [11]:
ygq=list(test['rate'])

# 生成提交文件
sub_path='./data/submission/'
sub=pd.read_csv(sub_path+'section2.txt', sep='\t')
sub['result'] = ygq
sub.to_csv('./data/section2DIN0.txt', sep='\t', index=False)

In [12]:
!nvidia-smi

Mon May 16 17:31:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.60.02    Driver Version: 510.60.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    On   | 00000000:41:00.0 Off |                  Off |
| 30%   36C    P2    64W / 230W |  23165MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------